In [1]:

import requests
import pandas as pd
import numpy as np

In [12]:
def get_masters_scores():
    url = "https://site.web.api.espn.com/apis/site/v2/sports/golf/leaderboard?league=pga"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        tournament = data['events'][0]['name']
        leaderboard = data['events'][0]['competitions'][0]['competitors']
        player_data = []
        for player in leaderboard:
            name = player['athlete']['displayName']
            # Fix names 
            if name == 'Byeong-Hun An': 
                name = 'Byeong Hun An'
            elif name == 'Cameron Davis': 
                name = 'Cam Davis'
            elif name == 'Ludvig Aberg': 
                name = 'Ludvig Åberg'
            elif name == 'Nicolai Højgaard': 
                name = 'Nicolai Hojgaard'
            elif name == 'Thorbjorn Olesen': 
                name = 'Thorbjørn Olesen'
            elif name == 'Joaquín Niemann': 
                name = 'Joaquin Niemann'
            
            position = player['status']['period']
            
            score = player['score']['displayValue']
            if score == 'E': 
                score = 0
            else: 
                score = int(score)
                
            player_data.append({'golfer_name': name, 'score': score})
        
        df = pd.DataFrame(player_data)
        df['score'] = df['score'].astype(int)
        return df
    else:
        return 'API Error'

def calculate_top_n(row, n):
    scores = [row['tier_1_1_score'], row['tier_1_2_score'], row['tier_1_3_score'], row['tier_2_1_score'], row['tier_2_2_score'], row['tier_2_3_score'], row['tier_3_2_score'], row['tier_4_1_score']]
    return sum(sorted(scores)[:n])


scores = get_masters_scores()

picks = pd.read_csv('masters_picks.csv')

# Merging golfers_df with masters_data_df on golfer names
for col in ['tier_1_1', 'tier_1_2', 'tier_1_3', 'tier_2_1', 'tier_2_2', 'tier_2_3', 'tier_3_1', 'tier_3_2', 'tier_4_1']: 
    if col == 'tier_1_1': 
        merged_df = pd.merge(picks, scores, how='left', left_on=col, right_on='golfer_name')
        merged_df = merged_df.drop(columns = 'golfer_name')
        merged_df = merged_df.rename(columns = {'score': f'{col}_score'})
    else: 
        merged_df = pd.merge(merged_df, scores, how ='left', left_on=col, right_on = 'golfer_name')
        merged_df = merged_df.drop(columns = ['golfer_name'])
        merged_df = merged_df.rename(columns = {'score': f'{col}_score'})

# Calculate top n scores
merged_df['top_6_score'] = merged_df.apply(lambda row: calculate_top_n(row, n=6), axis=1)
merged_df['top_7_score'] = merged_df.apply(lambda row: calculate_top_n(row, n=7), axis=1)
merged_df['top_8_score'] = merged_df.apply(lambda row: calculate_top_n(row, n=8), axis=1)

merged_df = merged_df.rename(columns = {'name': 'Name', 'tier_1_1': '1', 'tier_1_2': '2', 'tier_1_3': '3', 'tier_2_1': '4', 'tier_2_2': '5', 'tier_2_3': '6', 'tier_3_1': '7', 'tier_3_2': '8', 'tier_4_1': '9', 
                                        'tier_1_1_score': '1 Score', 'tier_1_2_score': '2 Score', 'tier_1_3_score': '3 Score', 'tier_2_1_score': '4 Score', 'tier_2_2_score': '5 Score', 'tier_2_3_score': '6 Score', 'tier_3_1_score': '7 Score', 'tier_3_2_score': '8 Score', 'tier_4_1_score': '9 Score',
                                        'top_6_score': 'Score', 'top_7_score': 'Tiebreak'})
for i in range(1, 10): 
    merged_df[f'Pick: {i}'] = merged_df[str(i)] + ' (' + merged_df[f'{i} Score'].astype(str) + ')'
merged_df['Rank'] = merged_df['Score'].rank(method='min').astype(int)
# Add blank col for spacing
merged_df[''] = ''

In [13]:
merged_df

,Name,1,2,3,4,5,6,7,8,9,...,Pick: 2,Pick: 3,Pick: 4,Pick: 5,Pick: 6,Pick: 7,Pick: 8,Pick: 9,Rank,
0,Scott Ario,Scottie Scheffler,Jordan Spieth,Joaquin Niemann,Tony Finau,Jason Day,Min Woo Lee,Justin Rose,Nicolai Hojgaard,Bubba Watson,...,Jordan Spieth (0),Joaquin Niemann (0),Tony Finau (0),Jason Day (0),Min Woo Lee (0),Justin Rose (0),Nicolai Hojgaard (0),Bubba Watson (0),1,
1,Jerico Johns,Scottie Scheffler,Jordan Spieth,Joaquin Niemann,Tony Finau,Jason Day,Min Woo Lee,Justin Rose,Nicolai Hojgaard,Bubba Watson,...,Jordan Spieth (0),Joaquin Niemann (0),Tony Finau (0),Jason Day (0),Min Woo Lee (0),Justin Rose (0),Nicolai Hojgaard (0),Bubba Watson (0),1,
2,Hank Workman,Rory McIlroy,Brooks Koepka,Patrick Cantlay,Dustin Johnson,Shane Lowry,Brian Harman,Russell Henley,Stephan Jaeger,Emiliano Grillo,...,Brooks Koepka (0),Patrick Cantlay (0),Dustin Johnson (0),Shane Lowry (0),Brian Harman (0),Russell Henley (0),Stephan Jaeger (0),Emiliano Grillo (0),1,
